In [ ]:
import numpy as np

class Neural_network:
    def __init__(self,input_neurons,hidden_neurons,output_neurons,learning_rate):
        """
        ニューラルネットワークの初期化
        """
        self.inneurons=input_neurons #入力層のニューロン数
        self.hneurons=hidden_neurons #隠れ層のニューロン数
        self.oneurons=output_neurons #出力層のニューロン数
        self.lr=learning_rate #学習率
        self.weight_initializer()
    
    def weight_initializer(self):
        self.w1=np.random.normal(
        0.0,
        pow(self.inneurons,-0.5),
        (self.hneurons,
        self.inneurons+1)
        )
        self.w2=np.random.normal(
                 0.0,
                 pow(self.hneurons,-0.5),
                 (self.oneurons,
                 self.hneurons+1)
                 )
        """
        self.w_o=np.random.normal(
                0.0,
                pow(self.hneurons,-0.5),
                (self.oneurons,
                self.hneurons+1)
                )
        """
        
    def sigmoid(self,x):#シグモイド
        return 1/(1+np.exp(-x))
    
    def softmax(self,x):
        c=np.max(x)
        exp_x=np.exp(x-c)#オーバーフロー防止
        sum_exp_x=np.sum(exp_x)
        y=exp_x/sum_exp_x
        return y
    
    def train(self,inputs_list,targets_list):
        """
        ニューラルネットワークの学習を行う
        ----
        inputs_list:訓練データの配列
        targets_list:正解ラベル
        """
        ##　入力層
        inputs=np.array(
                np.append(
                inputs_list,[1]),
                ndmin=2 #二次元化
                ).T
        ##隠れ層
        
        #入力層の出力に重み、バイアスをてきようして隠れ層に入力する
        hidden_inputs=np.dot(self.w1,inputs)
        
        #活性化関数を適用せて隠れ層からの出力
        hidden_outputs=self.sigmoid(hidden_inputs)
        #隠れ層の出力行列の末尾にバイアスのための[1]を追加
        hidden_outputs=np.append(
                        hidden_outputs,
                        [[1]],
                        axis=0)
        
        ## [出力層]
        #出力層への入力信号の計算
        final_inputs=np.dot(
                     self.w2,
                     hidden_outputs)
        
        # 活性化関数を適用して出力層から出力する
        final_outputs=self.softmax(final_inputs)
        
        ## バックプロパゲーション
        # 正解ラベルの配列を1列の行列に変換する
        targets=np.array(
        targets_list,
        ndmin=2).T
        
        #出力地と正解ラベルとの誤差
        output_errors=final_outputs -targets
        #出力値と正解ラベルとの誤差
        delta_output = output_errors*(1 - final_outputs)*final_outputs
        
        #重みを更新する前に隠れ層の出力誤差を求めておく
        hidden_errors=np.dot(
        self.w2.T,
        delta_output)
        
        # 出力層の重み、バイアスの更新
        self.w2-=self.lr*np.dot(
        #出力の誤差*(1-出力信号)*出力信号
        delta_output,
        # 隠れ層の出力行列の転置
        hidden_outputs.T)
        
        
        
        ##　バックプロパゲーション(隠れ層)
        
        #逆伝播された出力誤差からバイアスのものを取り除く
        hidden_errors_nobias=np.delete(
        hidden_errors,
        self.hneurons,
        axis=0
        )
        
        #隠れ層の出力行列からバイアスを除く
        hidden_outputs_nobias=np.delete(
        hidden_outputs,
        self.hneurons,
        axis=0)
        
        #隠れ層の重み,バイアスの更新
        self.w1-=self.lr*np.dot(
        hidden_errors_nobias*(1.0-hidden_outputs_nobias
                             )*hidden_outputs_nobias,
        #入力層の出力信号の行列を転置
        inputs.T
        )
    
    def evaluate(self,inputs_list):
        """
        学習した重みでテストデータを評価する
        """
        ##入力層
        inputs=np.array(
        np.append(inputs_list,[1]),
        ndmin=2).T
        
        ##隠れ層
        #入力層の出力に重み、バイアスをてきようして隠れ層に入力する
        hidden_inputs=np.dot(self.w1,inputs)
        #活性化関数を適用して隠れ層から出力する
        hidden_outputs=self.sigmoid(hidden_inputs)
        
        
        ##出力層
        final_inputs=np.dot(self.w2,
                           np.append(hidden_outputs,[1]),)
        
        #活性化関数を適用して出力層から出力する
        final_outputs=self.softmax(final_inputs)
        
        #出力層から出力を戻り値として返す
        return final_outputs

In [2]:
from keras.datasets import mnist
(x_trains,y_trains),(x_tests,y_tests)=mnist.load_data()

Using TensorFlow backend.


11493376/11490434 [==============================] - 78s 7us/step
